In [19]:
import mne
from mne.io import concatenate_raws

# 5,6,7,10,13,14 are codes for executed and imagined hands/feet
subject_id = 1
event_codes = [5,6,9,10,13,14]

# This will download the files if you don't have them yet,
# and then return the paths to the files.
#physionet_paths = mne.datasets.eegbci.load_data(subject_id, event_codes)
physionet_paths = ['C:\\Users\\Dansy\\Documents\\MasterThesis\\SampleBCI_2a\\A01T.gdf','C:\\Users\\Dansy\\Documents\\MasterThesis\\SampleBCI_2a\\A01E.gdf']
#print(physionet_paths)


# Load each of the files
parts = [mne.io.read_raw_edf(path, preload=True,stim_channel='auto', verbose='WARNING')
         for path in physionet_paths]

#print(parts)

# Concatenate them
raw = concatenate_raws(parts)

# Find the events in this dataset
events = mne.find_events(raw, shortest_event=0, stim_channel='STI 014')

# Use only EEG channels
eeg_channel_inds = mne.pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False,
                   exclude='bads')

# Extract trials, only using EEG channels
epoched = mne.Epochs(raw, events, None, tmin=1, tmax=4.1, proj=False, picks=eeg_channel_inds,
                baseline=None, preload=True)

Overlapping events detected. Use find_edf_events for the original events.
Interpolating stim channel. Events may jitter.
Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.


c:\users\dansy\appdata\local\programs\python\python36\lib\site-packages\mne\io\edf\edf.py:1035: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  etmode = np.fromstring(etmode, np.uint8).tolist()[0]
<ipython-input-19-944672f94dab>:17: RuntimeWarning: Overlapping events detected. Use find_edf_events for the original events.
  for path in physionet_paths]
<ipython-input-19-944672f94dab>:17: RuntimeWarning: Interpolating stim channel. Events may jitter.
  for path in physionet_paths]
<ipython-input-19-944672f94dab>:17: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  for path in physionet_paths]


Overlapping events detected. Use find_edf_events for the original events.
Interpolating stim channel. Events may jitter.
Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.


c:\users\dansy\appdata\local\programs\python\python36\lib\site-packages\mne\io\edf\edf.py:1035: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  etmode = np.fromstring(etmode, np.uint8).tolist()[0]
<ipython-input-19-944672f94dab>:17: RuntimeWarning: Overlapping events detected. Use find_edf_events for the original events.
  for path in physionet_paths]
<ipython-input-19-944672f94dab>:17: RuntimeWarning: Interpolating stim channel. Events may jitter.
  for path in physionet_paths]
<ipython-input-19-944672f94dab>:17: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  for path in physionet_paths]


Removing orphaned offset at the beginning of the file.
1169 events found
Events id: [  768  1537  1538  1539  1540  1551  1791  2560  2561  2562  2563  2574
 32766 33042 33043 33838]
1169 matching events found
Loading data for 1169 events and 776 original time points ...
0 bad epochs dropped


In [20]:
import numpy as np
# Convert data from volt to millivolt
# Pytorch expects float32 for input and int64 for labels.
X = (epoched.get_data() * 1e6).astype(np.float32)
y = (epoched.events[:,2] - 2).astype(np.int64) #2,3 -> 0,1

In [21]:
from braindecode.datautil.signal_target import SignalAndTarget

train_set = SignalAndTarget(X[:60], y=y[:60])
test_set = SignalAndTarget(X[60:], y=y[60:])

In [22]:
from braindecode.models.shallow_fbcsp import ShallowFBCSPNet
from torch import nn
from braindecode.torch_ext.util import set_random_seeds

# Set if you want to use GPU
# You can also use torch.cuda.is_available() to determine if cuda is available on your machine.
cuda = False
set_random_seeds(seed=20170629, cuda=cuda)
n_classes = 2
in_chans = train_set.X.shape[1]
# final_conv_length = auto ensures we only get a single output in the time dimension
model = ShallowFBCSPNet(in_chans=in_chans, n_classes=n_classes,
                        input_time_length=train_set.X.shape[2],
                        final_conv_length='auto').create_network()
if cuda:
    model.cuda()

ModuleNotFoundError: No module named 'torch'